# Read lagrangian trajectories and compute basic statistics

## Read data and get all pairs of particles

The data was generated from the fortran output of lagrangian trajectories "fort.1001" and converted into a netcdf file using the notebook "convert-data.ipynb"

In [ ]:
import numpy as np
import xarray as xr
import sys
sys.path.append('../')
from xdispersion import RelativeDispersion
import matplotlib.cm as cm
from matplotlib import cm
import matplotlib.pyplot as plt
import pandas as pd

save=False  #save figures or not
namedata = "ExpA" #ExpA or ExpAnomean
drifters = xr.open_dataset('./ExpA/fort.3001.np4000nt6000.nc')
#drifters = xr.open_dataset('./QGdaphne-E12e-8/fort.2001.np4000nt3000.nc')

FSLEfilename = "FSLEs/full_velocity/ExpA/fsles_expA.csv" #computed by Matthew
FSLEfilenamefluct = "FSLEs/no_mean_flow/ExpA/fsles_expA_fluct.csv" #computed by Matthew
fslefile_daphne="./Data_FLSE/FSLEData_ExpA.csv"

npa = drifters.dims['tracer'] #number particles
print(npa)
print(npa*(npa-1)/2) #expected number of pairs

Now we initialize a `RelativeDispersion` class, with the `drifters` dataset and associated names for position and velocity. 

In [ ]:
#rd = RelativeDispersion(drifters, ragged=False, ID='tracer',
#                        xpos='x', uvel='ux', time='time',
#                        ypos='y', vvel='uy', coord='cartesian')
rd = RelativeDispersion(drifters, ragged=False, ID='tracer',
                        xpos='x', uvel='ux', time='time',
                        ypos='y', vvel='uy', coord='polar')

print(rd)

pairs = rd.get_all_pairs()
pairs


## Plot a histogram of initial separation distances

In [ ]:
r0_min = pairs.r0.min().item()
r0_max = pairs.r0.max().item()
print(r0_min)
print(r0_max)

In [ ]:

fig, ax = plt.subplots(figsize=(6.5, 2.5), facecolor='w')

ax.hist(pairs.r0, bins=np.linspace(0, r0_max, 51), rwidth=0.86)
ax.set_xlabel('r0 (m)')
ax.set_title('histogram of initial separations')
ax.text(-0.1, 1.05, '(a)', transform=ax.transAxes, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## Select pairs based on initial separation

In [ ]:
origin_pairs = rd.get_original_pairs(pairs, r0=[0.005, 0.01])
# This is equivalent to:
# condition = np.logical_and(pairs.r0>=0.18, pairs.r0<=0.22)
# origin_pairs = pairs.where(condition, drop=True).astype(pairs.dtypes)

origin_pairs

## Calculate separation based measures

In [ ]:
from xdispersion import gen_rbins, rel_disp, rel_diff, kurtosis, cumul_inv_sep_time, famp_growth_rate, fsize_lyap_exp

alpha = 1.2
rbins = gen_rbins(0.01, 0.5, alpha)

# get all building blocks for the four measures. Here, rx is radial dispersion, ry means nothing and r is total dispersion
rx, ry, rxy, r, rpb = rd.separation_measures(origin_pairs)

r2   = rel_disp(r, order=2, mean_at='const-t') #order=2 so 2nd moment of separation
K2   = rel_diff(r, mean_at='const-t') #Relative diffusivity
K2r   = rel_diff(r, mean_at='const-r') #Relative diffusivity
Ku, lower, upper    = kurtosis(r, mean_at='const-t',ensemble=10) #kurtosis=4th moment
# Cumulative Inverse Separation Time
#CIST = cumul_inv_sep_time(r, rbins=rbins, lower=0.1, upper=0.9,
#                          maskout=[1e-8, 5e3], mean_at='const-r')

#Calculate same quantities for radial dispersion only 
r2x   = rel_disp(rx, order=2, mean_at='const-t') #order=2 so 2nd moment of separation
K2x   = rel_diff(rx, mean_at='const-t') #Relative diffusivity
K2rx   = rel_diff(rx, mean_at='const-r') #Relative diffusivity
Kux, lowerx, upperx    = kurtosis(rx, mean_at='const-t',ensemble=10) #kurtosis=4th moment

#Calculate finite amplitude growth rate (FAGR) and finite scale lyapunov exponent (FSLE)
FAGR, upper, lower = famp_growth_rate(r, mean_at='const-r', rbins=rbins, ensemble=10)
FSLE, upperFSLE, lowerFSLE = fsize_lyap_exp(r, rbins, mean_at='const-r',ensemble=10)
FAGRx, upper, lower = famp_growth_rate(rx, mean_at='const-r', rbins=rbins, ensemble=10)
FSLEx, upperFSLE, lowerFSLE = fsize_lyap_exp(rx, rbins, mean_at='const-r',ensemble=10)

### Physical fitted values for epsilon, kappa...

In [ ]:

if namedata=='ExpA':
    idx=0
elif namedata=='ExpB':
    idx=1
elif namedata=='ExpN':
    idx=2
elif namedata=='ExpO':
    idx=3
elif namedata=='ExpAnomean':
    idx=4
elif namedata=='ExpBnomean':
    idx=5
elif namedata=='ExpNnomean':
    idx=6   
elif namedata=='ExpOnomean':
    idx=7
elif namedata=='QG12': #a remesurer
    idx=8
elif namedata=='QG12nomean': #a remesurer
    idx=8       
    
#Values determined from fits: ExpA/B/N/O
betaall=[9e-3,8e-3,2.5e-3,8e-4,4.5e-3,4.5e-3,1.5e-3,4.3e-4,4.5e-3]
betaminall=[7e-3,6e-3,1.5e-3,6e-4,3e-3,3e-3,1e-3,3e-4,6e-3]
betamaxall=[13e-3,11e-3,4e-3,1e-3,7e-3,7e-3,2e-3,5.3e-4,9e-3]
betaxall=[2e-3,1.5e-3,0,1e-4,9e-4,9e-4,0,1.5e-4,2e-3]

kappalall=[5e-4,5e-4,1e-4,2e-5,1.8e-4,1.6e-4,3.5e-5,6e-6,1e-4]
kappalminall=[2e-4,2e-4,5e-5,1e-5,1e-4,1e-4,1.5e-5,4e-6,1e-4]
kappalmaxall=[8e-4,7e-4,2e-4,4e-5,3e-4,3e-4,6e-5,8e-6,1e-4]
kappalxall=[5e-5,3e-5,6e-6,1e-6,5e-5,3e-5,8e-6,1.3e-6,1e-4]
kappalxminall=[2e-5,1e-5,2e-6,5e-7,2e-5,1e-5,4e-6,0.5e-6,1e-4]
kappalxmaxall=[8e-5,5e-5,20e-6,2e-6,8e-5,6e-5,12e-6,2e-6,1e-4]

beta=betaall[idx] #8e-3 #1e-2 #3e-3 #8e-4
betamin=betaminall[idx]
betamax=betamaxall[idx]
betax=betaxall[idx] #radial dispersion

kappal=kappalall[idx] #8e-4 #6.5e-4 #1e-4 #3e-5 #6.5e-4 #total diffusivity
kappalmin=kappalminall[idx]
kappalmax=kappalmaxall[idx]
kappalx=kappalxall[idx] #radial diffusivity
kappalxmin=kappalxminall[idx]
kappalxmax=kappalxmaxall[idx]

Tall=[10,12,22,100,13,13,45,130,14]
T=Tall[idx] #seconds





### Plot dispersion (without and with radial component)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 5))  # (rows, cols)
ax = axes[0]  # pick the first subplot (dispersion)
#r2.plot.line(xscale='log', yscale='log', figsize=(6,3.5), marker='o', color='b', markersize=2, lw=0.5)
ax.scatter(r2.rtime,r2.values,marker='x',s=5,color='b')
ax.fill_between(r2.rtime[40:], 5.2675*(betamin)**3*r2.rtime[40:]**3, 5.2675*(betamax)**3*r2.rtime[40:]**3, color='r', alpha=0.15, edgecolor=None, label=None)
ax.plot(r2.rtime[40:],5.2675*(beta)**3*r2.rtime[40:]**3,color='r',label=fr'$5.2675 \epsilon t^3$ ($\epsilon$={beta**3:.1e} m$^{{2}}$s$^{{-3}}$)',linewidth=1.0)
ax.legend(loc='upper left')
ax.set_ylim(1e-5,max(r2.values)*100)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r"$t$ (s)")
ax.set_ylabel(r"$\langle r^2 \rangle$ (m$^2$)")
ax.set_title("Relative Dispersion")

# Second subplot (relative diffusivity)
#K2.plot(xscale='log', yscale='log', figsize=(6,3.5), marker='o', color='b', markersize=2, lw=0)
rtmp = np.sqrt(r2)
axes[1].scatter(rtmp.values,K2.values,marker='x',s=5,color='b')
# Plot shaded region
axes[1].fill_between(rtmp, 2.61*betamin*rtmp**(4/3), 2.61*betamax*rtmp**(4/3), color='r', alpha=0.15, edgecolor=None, label=None)
axes[1].fill_between(rtmp, rtmp*0+2*kappalmin, rtmp*0+2*kappalmax, color='k', alpha=0.1, edgecolor=None, label=None)
axes[1].plot(rtmp,2.61*beta*rtmp**(4/3),color='r',label=r'$2.61 \epsilon^{1/3} r^{4/3}$',linewidth=1.0)
axes[1].plot(rtmp,rtmp*0+2*kappal,'k',label=fr'$2\kappa$ ($\kappa$ = {kappal:.1e} m$^2$s$^{{-1}}$)',linewidth=1.0)
axes[1].set_title(r"Relative Diffusivity")
axes[1].set_ylim(ymin=1e-8)
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel(r"$r$ (m)")
axes[1].set_ylabel(r"$K_2$ (m$^2$s$^{{-1}}$)")
axes[1].legend()

# Third subplot (example: placeholder again)
#Ku.plot(xscale='linear', figsize=(6, 3), c='b')
axes[2].scatter(Ku.rtime, Ku.values, marker='x', s=5, color='b')
axes[2].plot(Ku.rtime,Ku.rtime*0+2,'--k')
axes[2].plot(Ku.rtime.values[:500], np.exp(8 * Ku.rtime.values[:500]/T), 'r-', label=fr'$\exp(8t/T)$ (T = {T:.2f} s)')
axes[2].legend()
axes[2].set_title("Kurtosis")
axes[2].set_xlabel(r"$t$ (s)")
axes[2].set_ylabel(r"$Ku$")
axes[2].set_ylim(0,max(Ku.values)*1.1)

# --- Add subplot labels (a), (b), (c) ---
fig.text(0.02, 0.86, "(a)", fontsize=12)
fig.text(0.34, 0.86, "(b)", fontsize=12)
fig.text(0.68, 0.86, "(c)", fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.93])  # leave space at top for labels
if save:
    output_path = f"../paper_daphne/basic-statistics_{namedata}.pdf"
    fig.savefig(output_path, format='pdf', bbox_inches='tight')  # save as PDF
plt.show()

################### ADD RADIAL DISPERSION ONLY ############################################


fig, axes = plt.subplots(1, 3, figsize=(12, 5))  # (rows, cols)
ax = axes[0]  # pick the first subplot (dispersion)
#r2.plot.line(xscale='log', yscale='log', figsize=(6,3.5), marker='o', color='b', markersize=2, lw=0.5)
ax.scatter(r2.rtime,r2.values,marker='x',s=5,color='b',label="full dispersion")
ax.scatter(r2x.rtime,r2x.values,marker='x',s=5,color='y',label="radial dispersion")
ax.plot(r2.rtime[40:],5.2675*(beta)**3*r2.rtime[40:]**3,color='r',label=fr'$5.2675 \epsilon t^3$ ($\epsilon$={beta**3:.1e} m$^{{2}}$s$^{{-3}}$)',linewidth=1.0)
ax.plot(r2.rtime[40:],5.2675*(betax)**3*r2.rtime[40:]**3,'--r',label=fr'$5.2675 \epsilon_\rho t^3$ ($\epsilon_\rho$={betax**3:.1e} m$^{{2}}$s$^{{-3}}$)',linewidth=1.0)
ax.legend(loc='upper left')
ax.set_ylim(1e-5,max(r2.values)*100)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r"$t$ (s)")
ax.set_ylabel(r"$\langle r^2 \rangle$ (m$^2$)")
ax.set_title("Relative Dispersion")

# Second subplot (relative diffusivity)
#K2.plot(xscale='log', yscale='log', figsize=(6,3.5), marker='o', color='b', markersize=2, lw=0)
rtmp = np.sqrt(r2)
axes[1].scatter(rtmp.values,K2x.values,marker='x',s=5,color='y')
axes[1].scatter(rtmp.values,K2.values,marker='x',s=5,color='b')
axes[1].fill_between(rtmp, rtmp*0+2*kappalxmin, rtmp*0+2*kappalxmax, color='k', alpha=0.1, edgecolor=None, label=None)
axes[1].plot(rtmp,2.61*beta*rtmp**(4/3),color='r',label=r'$2.61 \epsilon^{1/3} r^{4/3}$',linewidth=1.0)
axes[1].plot(rtmp,2.61*betax*rtmp**(4/3),'--r',label=r'$2.61 \epsilon_\rho^{1/3} r^{4/3}$',linewidth=1.0)
axes[1].plot(rtmp,rtmp*0+2*kappal,'k',label=fr'$2\kappa$ ($\kappa$ = {kappal:.1e} m$^2$s$^{{-1}}$)',linewidth=1.0)
axes[1].plot(rtmp,rtmp*0+2*kappalx,'--k',label=fr'$2\kappa_\rho$ ($\kappa_\rho$ = {kappalx:.1e} m$^2$s$^{{-1}}$)',linewidth=1.0)
axes[1].set_title(r"Relative Diffusivity")
axes[1].set_ylim(ymin=1e-8)
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel(r"$r$ (m)")
axes[1].set_ylabel(r"$K_2$ (m$^2$s$^{{-1}}$)")
axes[1].legend(loc='lower right')

# Third subplot (example: placeholder again)
#Ku.plot(xscale='linear', figsize=(6, 3), c='b')
axes[2].scatter(Kux.rtime, Kux.values, marker='x', s=5, color='y')
axes[2].scatter(Ku.rtime, Ku.values, marker='x', s=5, color='b')
axes[2].plot(Ku.rtime,Ku.rtime*0+2,'--k')
axes[2].plot(Ku.rtime.values[:500], np.exp(8 * Ku.rtime.values[:500]/T), 'r-', label=fr'$\exp(8t/T)$ (T = {T:.2f} s)')
axes[2].legend()
axes[2].set_title("Kurtosis")
axes[2].set_xlabel(r"$t$ (s)")
axes[2].set_ylabel(r"$Ku$")
axes[2].set_ylim(0,max(Ku.values)*1.1)

# --- Add subplot labels (a), (b), (c) ---
fig.text(0.02, 0.86, "(a)", fontsize=12)
fig.text(0.34, 0.86, "(b)", fontsize=12)
fig.text(0.68, 0.86, "(c)", fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.93])  # leave space at top for labels
if save:
    output_path = f"../paper_daphne/basic-statistics-rad_{namedata}.pdf"
    fig.savefig(output_path, format='pdf', bbox_inches='tight')  # save as PDF
plt.show()



In [ ]:

Ku.plot(xscale='linear', figsize=(6, 6), c='b')
#plt.fill_between(Ku.rtime, lower, upper, alpha=0.2, color='b')
plt.plot(Ku.rtime,Ku.rtime*0+2,'--k')
plt.plot(Kux.rtime,Kux.values,'y')
plt.xlim(0,20)
plt.ylim(0,20)
plt.plot(Ku.rtime.values[:300], np.exp(8 * Ku.rtime.values[:300]/T), 'r-', label=f'exp(8*t/T), T = {T:.2f}s')
plt.legend()

### Compute pFSLE

We can also calculate the pFSLE from the relative diffusivity (see eqaution 2.8 in Lacasce and Meunier JFM 2021)

In [ ]:
pFSLE=K2.values/rtmp.values**2 #this has indeed dimension of inverse time
pFSLE2=K2r.values/K2r.rbin.values**2
pFSLEx=K2x.values/rtmp.values**2 #this has indeed dimension of inverse time
pFSLEx2=K2rx.values/K2rx.rbin.values**2

# Digitize rtmp values into bins
bin_indices = np.digitize(rtmp.values, rbins)
# Prepare arrays to store binned statistics
# Prepare array to store binned mean
pFSLE_binned_mean = np.full(len(rbins[1:]), np.nan)
pFSLEx_binned_mean = np.full(len(rbins[1:]), np.nan)

# Compute mean pFSLE in each bin
for i in range(1, len(rbins)):
    mask = bin_indices == i
    if np.any(mask):
        pFSLE_binned_mean[i - 1] = np.nanmean(pFSLE[mask])
        pFSLEx_binned_mean[i - 1] = np.nanmean(pFSLEx[mask])

plt.figure(figsize=(5, 6))
#plt.scatter(rtmp.values,pFSLE,marker='x',color='red')
plt.scatter(rtmp.values, pFSLE, marker='x', color='red', alpha=0.3, label='Raw data')
plt.plot(rbins[1:], pFSLE_binned_mean, 'o-', color='black', label='Binned mean')
plt.scatter(rtmp.values, pFSLEx, marker='x', color='blue', alpha=0.3, label='Raw data (rad)')
plt.plot(rbins[1:], pFSLEx_binned_mean, 'o-', color='black', label='Binned mean (rad)')
plt.plot(rtmp, 4*kappal*np.log(2)/(alpha**2-1)*rtmp**(-2), 'r-', label=r'$4\kappa_L \ln(2)/( \alpha^2 - 1)  r^{-2}$')
plt.plot(rtmp, 4*beta*2.6741/(9*(alpha**(2/3)-1))*rtmp**(-2/3), 'k-', label=r'$4\beta \cdot 2.6741 / [9 (\alpha^{2/3} - 1)]  r^{-2/3}$')
plt.plot(rtmp[:5], rtmp[:5]*0+2/T/np.log(alpha), 'y-',label=r'$2 / (T \ln(\alpha))$')
plt.plot(K2r.rbin.values,pFSLE2)
plt.xscale('log')
plt.yscale('log')
plt.legend()

## Compute the PDF and CDF and plot them

In [ ]:
from xdispersion import * 

PDF=prob_dens_func(r,rbins)
CDF = cumul_dens_func(PDF,rbins)

PDFx=prob_dens_func(rx,rbins)
CDFx = cumul_dens_func(PDFx,rbins)

In [ ]:


# Select fixed rbin values
rbins_to_plot = [rbins.values[8],rbins.values[11], rbins.values[15], rbins.values[17], rbins.values[19]]  # example rbin values

# Choose a colormap and get distinct colors
colors = cm.viridis(np.linspace(0, 1, len(rbins_to_plot)))

plt.figure(figsize=(7, 5))

for i, rb in enumerate(rbins_to_plot):
    # Select slices
    cdf_slice = CDF.sel(rbin=rb, method='nearest')
    
    # Get actual rbin (in case nearest doesn't match exactly)
    rbin_val = cdf_slice.rbin.values

    # Plot CDF vs time at this rbin
    plt.plot(
        CDF.rtime, cdf_slice,
        color=colors[i],
        linestyle='--',
        marker='x',
        label=f'rbin = {rbin_val:.2f} m'
    )

plt.plot(CDF.rtime,CDF.rtime*0+0.5,color='black',linestyle='--')
plt.xlabel(r'$t$ (s)')
plt.ylabel('CDF')
plt.title('CDF vs Time at selected rbin values')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


## Plot the fit of the CDF to find the values for which c=0.5

### Define exponential fit

In [ ]:

from typing import Tuple
from scipy.optimize import curve_fit
import numpy as np

def exp_decay_offset_fit(t: np.array, y: np.array) -> Tuple[float, float, float, float, float]:
    """Fit y = A * exp(-(t - t0) / tau) + C with delay and offset."""
    idx = ~np.isnan(y)
    t_fit = t[idx]
    y_fit = y[idx]

    if len(y_fit) < 4:
        return np.nan, np.nan, np.nan, np.nan, np.nan

    try:
        def model(t, A, tau, t0, C):
            t_shifted = t - t0
            return np.where(t_shifted >= 0, A * np.exp(-t_shifted / tau) + C, A + C)

        # Initial guesses: [A, tau, t0, C]
        A0 = y_fit.max() - y_fit.min()
        tau0 = (t_fit.max() - t_fit.min()) / 3
        t0_0 = t_fit.min()
        C0 = y_fit.min()
       
        # Bounds: A free, tau>0, t0>=0, C free
        lower_bounds = [-np.inf, 1e-12, 0, -np.inf]
        upper_bounds = [ np.inf,  np.inf, np.inf,  np.inf]
        
        popt, _ = curve_fit(model, t_fit, y_fit, p0=[A0, tau0, t0_0, C0],bounds=(lower_bounds, upper_bounds), maxfev=10000)
        A, tau, t0, C = popt


        y_pred = model(t_fit, A, tau, t0, C)
        rmse = np.sqrt(np.mean((y_pred - y_fit) ** 2))
        return A, tau, t0, C, rmse

    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan



### Apply the exponential fit 

In [ ]:
#A_exp, tau_exp, t0_exp, C_exp, rmse_exp = xr.apply_ufunc(
#    exp_decay_offset_fit,
#    CDF['rtime'],
#    CDF,
#    dask='allowed',
#    input_core_dims=[['rtime'], ['rtime']],
#    output_core_dims=[[], [], [], [], []],
#    vectorize=True
#)

# Exclude the first rtime point for fitting
CDF_fit = CDF.isel(rtime=slice(1, None))
rtime_fit = CDF_fit['rtime']

#plt.plot(rtime_fit, CDF_fit.sel(rbin=0.042998, method='nearest'), '.', markersize=1, label='Data (before fit)')

# Apply the fitting function on total dispersion
A_exp, tau_exp, t0_exp, C_exp, rmse_exp = xr.apply_ufunc(
    exp_decay_offset_fit,
    rtime_fit,
    CDF_fit,
    dask='allowed',
    input_core_dims=[['rtime'], ['rtime']],
    output_core_dims=[[], [], [], [], []],
    vectorize=True
)

# Apply the fitting function on radial dispersion
A_expx, tau_expx, t0_expx, C_expx, rmse_expx = xr.apply_ufunc(
    exp_decay_offset_fit,
    CDFx['rtime'],
    CDFx,
    dask='allowed',
    input_core_dims=[['rtime'], ['rtime']],
    output_core_dims=[[], [], [], [], []],
    vectorize=True
)


### Apply semilogfit

In [ ]:
from xdispersion import semilog_fit

lower=0.1
upper=0.9
CDFrng = CDF.where(np.logical_and(CDF>lower, CDF<upper))

# Use a semilog fit to find where CDF crosses value 0.5         
slope, inter, rms = xr.apply_ufunc(semilog_fit, CDFrng['rtime'], CDFrng,
                                               dask='allowed',
                                               input_core_dims=[['rtime'], ['rtime']],
                                               output_core_dims=[[], [], []],
                                               vectorize=True)
            
t_half_semilog = np.exp((0.5 - inter) / slope)
diff = t_half_semilog.diff('rbin')
CIST  = 1.0 / diff





### COmpare semilog and exponential fits

In [ ]:
plt.figure(figsize=(9, 6))

for i, rb in enumerate(rbins_to_plot):
    # Raw CDF
    cdf_line = CDF.sel(rbin=rb, method='nearest')
    rtime_vals = cdf_line['rtime'].values
    cdf_vals = cdf_line.values
    rb_val = cdf_line.rbin.values

    plt.plot(rtime_vals, cdf_vals, 'o', color=colors[i], label=f'CDF, rbin={rb_val:.2f}')

    # Semi-log fit
    m = slope.sel(rbin=rb, method='nearest').values
    b = inter.sel(rbin=rb, method='nearest').values
    if np.isfinite(m) and np.isfinite(b):
        log_rtime = np.log(rtime_vals)
        semi_log_fit = m * log_rtime + b
        plt.plot(rtime_vals, semi_log_fit, '--', color=colors[i], label=f'Semi-log Fit, rbin={rb_val:.2f}')

    # Exponential fit
    A = A_exp.sel(rbin=rb, method='nearest').values
    tau = tau_exp.sel(rbin=rb, method='nearest').values
    t0 = t0_exp.sel(rbin=rb, method='nearest').values
    C = C_exp.sel(rbin=rb, method='nearest').values

    if np.all(np.isfinite([A, tau, t0, C])):
        t_vals = rtime_vals
        t_shifted = t_vals - t0
        exp_fit = np.where(t_shifted >= 0, A * np.exp(-t_shifted / tau) + C, A + C)
    
        #plt.plot(t_vals, exp_fit, ':', color=colors[i], label=f'Exp Fit (offset), rbin={rb_val:.2f}')
        plt.plot(t_vals, exp_fit, ':', color='black', label=f'Exp Fit (offset), rbin={rb_val:.2f}')


plt.xlabel('Time')
plt.ylabel('CDF')
plt.title('CDF with Semi-log and Exponential Fits')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Plot CDF with exponential fit for paper and save figure

In [ ]:
fig=plt.figure(figsize=(7, 5))

for i, rb in enumerate(rbins_to_plot):
    # Raw CDF
    cdf_line = CDF_fit.sel(rbin=rb, method='nearest')
    rtime_vals = cdf_line['rtime'].values
    cdf_vals = cdf_line.values
    rb_val = cdf_line.rbin.values

    plt.plot(rtime_vals, cdf_vals, 'x', markersize=3, color=colors[i], label=f'rbin={rb_val:.2f} m')

    # Exponential fit
    A = A_exp.sel(rbin=rb, method='nearest').values
    tau = tau_exp.sel(rbin=rb, method='nearest').values
    t0 = t0_exp.sel(rbin=rb, method='nearest').values
    C = C_exp.sel(rbin=rb, method='nearest').values

    if np.all(np.isfinite([A, tau, t0, C])):
        t_vals = rtime_vals
        t_shifted = t_vals - t0
        exp_fit = np.where(t_shifted >= 0, A * np.exp(-t_shifted / tau) + C, A + C)
    
        label_exp = f'Exponential fits' if i == len(rbins_to_plot)-1 else None
        plt.plot(t_vals, exp_fit, '--', color='black', label=label_exp)

plt.plot(CDF.rtime,CDF.rtime*0+0.5,color='black',linestyle=':')
plt.xlabel('t (s)')
plt.ylabel('CDF')
plt.title('CDF with Exponential Fits')
plt.legend()
plt.grid(True)
plt.tight_layout()

if save:
    output_path = f"../paper_daphne/CDF_{namedata}.pdf"
    fig.savefig(output_path, format='pdf', bbox_inches='tight')  # save as PDF
plt.show()

### Plot CDF fit for radial dispersion

In [ ]:
fig=plt.figure(figsize=(7, 5))


for i, rb in enumerate(rbins_to_plot):
    # Raw CDF
    cdf_line = CDFx.sel(rbin=rb, method='nearest')
    rtime_vals = cdf_line['rtime'].values
    cdf_vals = cdf_line.values
    rb_val = cdf_line.rbin.values

    plt.plot(rtime_vals, cdf_vals, 'x', markersize=3, color=colors[i], label=f'rbin={rb_val:.2f} m')

    # Exponential fit
    A = A_expx.sel(rbin=rb, method='nearest').values
    tau = tau_expx.sel(rbin=rb, method='nearest').values
    t0 = t0_expx.sel(rbin=rb, method='nearest').values
    C = C_expx.sel(rbin=rb, method='nearest').values

    if np.all(np.isfinite([A, tau, t0, C])):
        t_vals = rtime_vals
        t_shifted = t_vals - t0
        exp_fit = np.where(t_shifted >= 0, A * np.exp(-t_shifted / tau) + C, A + C)
    
        label_exp = f'Exponential fits' if i == len(rbins_to_plot)-1 else None
        plt.plot(t_vals, exp_fit, '--', color='black', label=label_exp)

plt.plot(CDFx.rtime,CDFx.rtime*0+0.5,color='black',linestyle=':')
plt.xlabel('t (s)')
plt.ylabel('CDF')
plt.title('CDF with Exponential Fits')
plt.legend()
plt.grid(True)
plt.tight_layout()

if save:
    output_path = f"../paper_daphne/CDF_radial_{namedata}.pdf"
    fig.savefig(output_path, format='pdf', bbox_inches='tight')  # save as PDF
plt.show()

### Compute half time from exponential fit

In [ ]:
# Compute t₀.₅ (time where CDF = 0.5)
valid_mask = np.logical_and(
    A_exp > 0,
    np.logical_and(tau_exp > 0, (0.5 - C_exp) / A_exp > 0)
)
valid_maskx = np.logical_and(
    A_expx > 0,
    np.logical_and(tau_expx > 0, (0.6 - C_expx) / A_expx > 0)
)

t_half_exp = xr.where(
    valid_mask,
    t0_exp - tau_exp * np.log((0.5 - C_exp) / A_exp),
    np.nan
)
t_half_expx = xr.where(
    valid_maskx,
    t0_expx - tau_expx * np.log((0.6 - C_expx) / A_expx),
    np.nan
)

# Compute Δt_half and its inverse
dt_half = t_half_exp.diff('rbin')
dt_halfx = t_half_expx.diff('rbin')
CIST_exp = 1.0 / dt_half
CIST_expx = 1.0 / dt_halfx

# rbin values corresponding to upper bins in diff
rbin_upper = t_half_exp['rbin'][1:]

### Plot the half time, and its inverse of the difference

In [ ]:
# Find the index of the value in the CDF nearest to 0.5 (to compare with the fit)
idx_nearest = abs(CDF - 0.5).argmin(dim='rtime')
t_half_actual = CDF['rtime'].isel(rtime=idx_nearest)

# Find rbin_max where t_half_actual is closest to tmax 
tmax = CDF.rtime.values.max()
tol = 0.1 * tmax # Define tolerance
mask = np.abs(t_half_actual - tmax) <= tol # Mask rbin values within ±5% of tmax
rbin_candidates = CDF.rbin.where(mask, drop=True) # Get the smallest rbin that satisfies the condition
if rbin_candidates.size > 0:
    rbin_max = rbin_candidates.min().item()
else:
    rbin_max = CDF.rbin.max().item()

# Compute inverse of time differences (Δt⁻¹)
inv_diff_actual = 1.0 / t_half_actual.diff('rbin')
inv_diff_semilog = 1.0 / t_half_semilog.diff('rbin')
inv_diff_exp = 1.0 / t_half_exp.diff('rbin')
inv_diff_expx = 1.0 / t_half_expx.diff('rbin')
# Use the *upper* bin of each pair for x-axis
#rbin_upper = CDF.rbin.isel(rbin=slice(1, None))

fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharex=True)

# --- Subplot 1: Time where CDF = 0.5 ---
axs[0].plot(CDF.rbin, t_half_actual, '+', label='Actual (nearest CDF=0.5)')
axs[0].plot(CDF.rbin, t_half_semilog, 'x', label='Fitted (semilog)')
axs[0].plot(CDF.rbin, t_half_exp, 'x', label='Fitted (offset exp)')
axs[0].plot(CDFx.rbin, t_half_expx, 'x', label='Fitted radial (offset exp)')
axs[0].plot([0.005,1],[tmax,tmax],color='k')
axs[0].axvspan(rbin_max, axs[0].get_xlim()[1], color='gray', alpha=0.3) # Shade rbin > rbin_max
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].set_xlabel('rbin (m)')
axs[0].set_ylabel('Time (CDF=0.5) (s)')
axs[0].legend()
axs[0].grid(True)
axs[0].set_title('Time Comparison (Actual vs Fitted)')
axs[0].set_ylim(0,10**4)

# --- Subplot 2: Inverse Time Differences ---
axs[1].plot(rbin_upper, inv_diff_actual, '+', label='1/Δt actual')
axs[1].plot(rbin_upper, inv_diff_semilog, 'x', label='1/Δt semilog')
axs[1].plot(rbin_upper, inv_diff_exp, 'x', label='1/Δt exp')
axs[1].plot(rbin_upper, inv_diff_expx, 'x', label='1/Δt exp radial')
axs[1].plot(rbin_upper, 4*kappal*np.log(2)/(alpha**2-1)*rbin_upper**(-2), 'r-', label=r'Taylor')
axs[1].plot(rbin_upper, 4*beta*2.6741/(9*(alpha**(2/3)-1))*rbin_upper**(-2/3), 'k-', label=r'Richardson')
axs[1].axvspan(rbin_max, axs[1].get_xlim()[1], color='gray', alpha=0.3) # Shade rbin > rbin_max
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel('rbin (m)')
axs[1].set_ylabel('1 / Δtime (s$^{-1}$)')
axs[1].legend()
axs[1].grid(True)
axs[1].set_title('Inverse of Time Differences (aligned to upper rbin)')
axs[1].set_ylim(10**(-4),10)

plt.tight_layout()
plt.show()

### Compare CIST using semilogfit and exponential fit

In [ ]:


# Plots

plt.figure(figsize=(5, 6))
plt.plot(rbin_upper, CIST_exp, 'bo', label='CIST exp')
plt.plot(rbin_upper, CIST_expx, 'go', label='CIST exp radial')
plt.plot(rbin_upper, inv_diff_actual, '+', label='1/Δt actual')
CIST.plot.scatter(yscale='log',xscale='log', c='k', label='CIST semilog')
plt.plot(rbins[5:], 4*kappal*np.log(2)/(alpha**2-1)*rbins[5:]**(-2), 'r-', label=r'$4\kappa_L \ln(2)/( \alpha^2 - 1)  r^{-2}$')
plt.plot(rbin_upper[1:15], 4*beta*2.6741/(9*(alpha**(2/3)-1))*rbin_upper[1:15]**(-2/3), 'g-', label=r'$4\beta \cdot 2.6741 / [9 (\alpha^{2/3} - 1)]  r^{-2/3}$')
plt.plot(rbin_upper[:5], rbin_upper[:5]*0+2/T/np.log(alpha), 'k-',label=r'$2 / (T \ln(\alpha))$')
plt.plot(rbins[1:], pFSLE_binned_mean, 'x', color='k', label='pFSLE')
plt.axvspan(rbin_max, axs[0].get_xlim()[1], color='gray', alpha=0.3) # Shade rbin > rbin_max
plt.xscale('log')
plt.yscale('log')
plt.xlabel('rbin')
plt.ylabel('1 / Δt₀.₅')
plt.ylim(10**(-3), 100)
plt.xlim(rbins.min(),1.1*rbins.max())
plt.title('Inverse Δt₀.₅ vs rbin (filtered, CDF(t=0) ≥ 0.5)')
plt.grid(True, which='both')
plt.legend()
plt.tight_layout()
plt.show()



### Final CIST plot, incorporating FLSE data from Matthew

Firstimport Matthew's data and data from my PhD manuscript

In [ ]:
# Import the CSV file
fsle = pd.read_csv(FSLEfilename)
fslef = pd.read_csv(FSLEfilenamefluct)

# Check the first few rows to verify column indices
print(fsle.head())





In [ ]:

fsled = pd.read_csv(fslefile_daphne)
print(fsled.head())

# Plot column 2 vs column 4
# Note: pandas uses 0-based indexing, so column 2 = index 1, column 4 = index 3
plt.figure(figsize=(5,4))
plt.plot(fsle.iloc[:, 1], fsle.iloc[:, 3], 'x', color='blue',label="Matthew") 
plt.plot(fslef.iloc[:, 1], fslef.iloc[:, 3], '.', color='blue',label="Matthew fluct") 
plt.plot(fsled.iloc[:, 4], fsled.iloc[:, 5], 'x', color='red',label="Daphne") 
plt.plot(fsled.iloc[:, 0], fsled.iloc[:,1], '.', color='red',label="Daphne rad") 
plt.xlabel("rbin (m)")
plt.ylabel("FSLE (1/s)")
plt.grid(True)
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.tight_layout()
plt.show()


In [ ]:
#kappal=5e-4

fig, axs = plt.subplots(1, 2, figsize=(8, 5), sharex=False)

axs[0].grid(True, which='both',linewidth=0.5, color='lightgrey')
axs[0].plot(rbin_upper[rbin_upper<=rbin_max], CIST_exp[rbin_upper<=rbin_max], 'xk', markeredgewidth=1.5, label='CIST',zorder=4)
axs[0].plot(rbins[1:], pFSLE_binned_mean, 'o', color='k',  markersize=4, label='pFSLE',zorder=3)
#axs[0].plot(FSLE.rbin,FSLE.values,'o',color='blue',markersize=4,label='FSLE')
axs[0].plot(FAGR.rbin,FAGR.values,'d',markersize=3, color=(0.6, 0.6, 0.6),label='FAGR')

#Matthew FSLE data
#axs[0].plot(fsle.iloc[:, 1], fsle.iloc[:, 3]/fsle.iloc[1, 3]*pFSLE_binned_mean[0], 'o', markersize=2, color=(0.5, 0.5, 0.5), label='FSLE',zorder=2) 
#axs[0].plot(fslef.iloc[:, 1], fslef.iloc[:, 3]/fslef.iloc[1, 3]*pFSLE_binned_mean[0], 'o', markersize=2, color=(0.5, 0.5, 0.5), label='FSLE',zorder=2) 
#Daphne FSLE data
#axs[0].plot(fsled.iloc[:, 4], fsled.iloc[:, 5]/fsled.iloc[1,5]*pFSLE_binned_mean[0], 'o', markersize=2, color=(0.5, 0.5, 0.5), label="FSLE",zorder=2) 
axs[0].plot(rbins[2:22], 4*kappal*np.log(2)/(alpha**2-1)*rbins[2:22]**(-2), 'r-', label=r'$4\kappa \frac{\ln(2)}{ \alpha^2 - 1}  r^{-2}$',linewidth=1.0)
#axs[0].plot(rbins[2:22], 2*kappal*rbins[2:22]**(-2), 'r-', label=r'$4\kappa \frac{\ln(2)}{ \alpha^2 - 1}  r^{-2}$',linewidth=1.0)
axs[0].plot(rbin_upper[0:17], 4*beta*2.6741/(9*(alpha**(2/3)-1))*rbin_upper[0:17]**(-2/3), 'y-', label=r'$4\epsilon^{1/3}  \frac{2.6741}{ 9 (\alpha^{2/3} - 1)}  r^{-2/3}$',linewidth=1.0)
#axs[0].plot(rbin_upper[0:17], 2.6099*beta*rbin_upper[0:17]**(-2/3), 'y-', label=r'$4\epsilon^{1/3}  \frac{2.6741}{ 9 (\alpha^{2/3} - 1)}  r^{-2/3}$',linewidth=1.0)
axs[0].plot(rbin_upper[:5], rbin_upper[:5]*0+2/T/np.log(alpha), 'k-',label=r'$2 / (T \ln(\alpha))$',linewidth=1.0)
axs[0].axvspan(rbin_max, axs[0].get_xlim()[1], color='gray', alpha=0.3) # Shade rbin > rbin_max
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].set_xlabel('rbin (m)')
axs[0].set_ylabel(r'CIST and FLSE (s$^{-1}$)')
axs[0].set_ylim(10**(-3), 2)
axs[0].set_xlim(rbins.min(),1.1*rbins.max())
axs[0].set_title(f"{namedata} total dispersion")
axs[0].legend(loc='lower left')

axs[1].grid(True, which='both',linewidth=0.5, color='lightgrey')
axs[1].plot(rbin_upper, CIST_expx, 'xk', markeredgewidth=1.5, label='CIST',zorder=4)
axs[1].plot(rbins[1:], pFSLEx_binned_mean, 'o', color='k', markersize=4, label='pFSLE',zorder=3)
#axs[1].plot(FSLEx.rbin,FSLEx.values,'o',color='blue',markersize=4,label='FSLE')
axs[1].plot(FAGRx.rbin,FAGRx.values,'d',markersize=3, color=(0.6, 0.6, 0.6),label='FAGR')
#Daphne FSLE data
#axs[1].plot(fsled.iloc[:, 0], fsled.iloc[:, 1]/fsled.iloc[1,1]*pFSLEx_binned_mean[0], 'o', markersize=2, color=(0.5, 0.5, 0.5), label="FSLE",zorder=2) 
#axs[1].plot(rbins[0:5], 4*betax*2.6741/(9*(alpha**(2/3)-1))*rbins[0:5]**(-2/3), 'y-', label=r'$4\epsilon_\rho^{1/3}  \frac{2.6741}{ 9 (\alpha^{2/3} - 1)}  r^{-2/3}$',linewidth=1.0)
axs[1].plot(rbins[1:17], 4*kappalx*np.log(2)/(alpha**2-1)*rbins[1:17]**(-2), 'r-', label=r'$4\kappa_\rho \frac{\ln(2)}{ \alpha^2 - 1}  r^{-2}$',linewidth=1.0)
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel('rbin (m)')
axs[1].set_ylabel(r'CIST and FLSE (s$^{-1}$)')
axs[1].set_ylim(10**(-3.5), 2)
axs[1].set_xlim(rbins.min(),0.2)
axs[1].set_title(f"{namedata} radial dispersion")
axs[1].legend(loc='upper right')

save=True
plt.tight_layout()
if save:
    output_path = f"../paper_daphne/CISTandFAGR_{namedata}.pdf"
    fig.savefig(output_path, format='pdf', bbox_inches='tight')  # save as PDF
plt.show()